In [115]:
# Importing modules that will be used in this notebook
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading the Data

## Loading the First Dataset

In [116]:
# reading in data from using pandas read_csv function
from pandas import DataFrame


charity_df = pd.read_csv('https://www.irs.gov/pub/irs-soi/eo_ky.csv')

# displaying the top 5 rows of the newly created comparsion_df DataFrame to ensure it was loaded correctly
charity_df.head()

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
0,10356732,CARRIAGE MUSEUM OF AMERICA,NaN,4075 IRON WORKS PKWY,LEXINGTON,KY,40511-8483,0,3,3,...,6,3,1,0,12,1476492.0,58555.0,57604.0,A540,AT THE KENTUCKY HORSE PARK
1,10551398,HIGHLANDS BAND ASSOCIATION INC,% STANLEY ROSS,PO BOX 75029,FORT THOMAS,KY,41075-0029,0,3,3,...,0,0,2,0,5,0.0,0.0,0.0,N60,NaN
2,10554843,JEFFERSON COUNTY SEARCH DOG ASSOCIATION,% HOLLY HATFIELD,8004 SMYRNA PKWY,LOUISVILLE,KY,40228-1808,0,3,3,...,0,0,2,0,12,0.0,0.0,0.0,P80,NaN
3,10557144,AUGUSTA ART GUILD,% PRESIDENT,116 MAIN ST,AUGUSTA,KY,41002-1035,0,3,3,...,0,0,2,0,12,0.0,0.0,0.0,A40,NaN
4,10557631,KENTUCKY ALPACA ASSOCIATION INC,%MARTHA SANDERS,134 N LUCAS RD,GLASGOW,KY,42141-8778,0,5,3,...,0,0,2,0,12,0.0,0.0,0.0,K26,KY CLASSIC ALPACA SHOW


## Loading the Second Dataset

In [117]:
income_df = pd.read_csv('Household_Income_by_Zip.csv')
income_df.head()

,GEO_ID,NAME,S1901_C01_001E,S1901_C01_012E,S1901_C01_013E
0,Geography,Geographic Area Name,Estimate!!Households!!Total,Estimate!!Households!!Median income (dollars),Estimate!!Households!!Mean income (dollars)
1,8600000US38079,ZCTA5 38079,1311,32629,54397
2,8600000US40003,ZCTA5 40003,749,78068,84259
3,8600000US40004,ZCTA5 40004,11732,61974,76107
4,8600000US40006,ZCTA5 40006,1675,61838,67429


# Cleaning the Data

## Cleaning the First Dataset

In [118]:
# dropping the columns I will not need for this project
charity_df = charity_df.drop(['ICO', 'ORGANIZATION', "SUBSECTION", "CLASSIFICATION", "PF_FILING_REQ_CD","FILING_REQ_CD", "ASSET_CD", "INCOME_CD"], axis=1)

#changing the Ruling Date column to a date
charity_df['RULING'] = pd.to_datetime(charity_df['RULING'])

# looking at the type of each column to make sure they are correct
charity_df.dtypes


EIN                       int64
NAME                     object
STREET                   object
CITY                     object
STATE                    object
ZIP                      object
GROUP                     int64
AFFILIATION               int64
RULING           datetime64[ns]
DEDUCTIBILITY             int64
FOUNDATION                int64
ACTIVITY                  int64
STATUS                    int64
TAX_PERIOD              float64
ACCT_PD                   int64
ASSET_AMT               float64
INCOME_AMT              float64
REVENUE_AMT             float64
NTEE_CD                  object
SORT_NAME                object
dtype: object

In [119]:
# adding a column for NTEE Common Codes using the column of specific NTEE Code
charity_df['NTEE_COMMON_CODE'] = charity_df.NTEE_CD.str[:1]

# Replacing the NTEE codes with the code description

# Making a dictionary of the NTEE Common Codes and their descriptions
Common_Codes = {
    "A" : "Arts & Culture",
    "B" : "Education",
    "C" : "Environment",
    "D" : "Animals",
    "E" : "Health",
    "F" : "Mental Health",
    "G" : "Diseases & Disorders",
    "H" : "Medical Research",
    "I" : "Legal Related",
    "J" : "Job Related",
    "K" : "Food & Nutrition",
    "L" : "Housing & Shelter",
    "M" : "Public Safety & Disaster Relief",
    "N" : "Sports & Leisure",
    "O" : "Youth Development",
    "P" : "Human Services",
    "Q" : "Foreign Affairs",
    "R" : "Civil Rights",
    "S" : "Community Improvement",
    "T" : "Philanthropy & Voluntarism",
    "U" : "Science & Technology Research",
    "V" : "Social Science Research",
    "W" : "Public, Society Benefit",
    "X" : "Religon",
    "Y" : "Memebership Based",
    "Z" : "Unknown"
    }

charity_df['NTEE_COMMON_CODE'].replace(Common_Codes, inplace=True)

In [120]:
# Splitting the Zip Column into 2 columns - one for the Zip and one for the +4

charity_df[['ZIP', 'ZIP+4']] = charity_df['ZIP'].str.split('-', 1, expand=True)
charity_df.head()

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,AFFILIATION,RULING,DEDUCTIBILITY,...,STATUS,TAX_PERIOD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,NTEE_COMMON_CODE,ZIP+4
0,10356732,CARRIAGE MUSEUM OF AMERICA,4075 IRON WORKS PKWY,LEXINGTON,KY,40511,0,3,1970-01-01 00:00:00.000197808,1,...,1,202012.0,12,1476492.0,58555.0,57604.0,A540,AT THE KENTUCKY HORSE PARK,Arts & Culture,8483
1,10551398,HIGHLANDS BAND ASSOCIATION INC,PO BOX 75029,FORT THOMAS,KY,41075,0,3,1970-01-01 00:00:00.000200203,1,...,1,201905.0,5,0.0,0.0,0.0,N60,NaN,Sports & Leisure,0029
2,10554843,JEFFERSON COUNTY SEARCH DOG ASSOCIATION,8004 SMYRNA PKWY,LOUISVILLE,KY,40228,0,3,1970-01-01 00:00:00.000200204,1,...,1,202112.0,12,0.0,0.0,0.0,P80,NaN,Human Services,1808
3,10557144,AUGUSTA ART GUILD,116 MAIN ST,AUGUSTA,KY,41002,0,3,1970-01-01 00:00:00.000201407,1,...,1,202112.0,12,0.0,0.0,0.0,A40,NaN,Arts & Culture,1035
4,10557631,KENTUCKY ALPACA ASSOCIATION INC,134 N LUCAS RD,GLASGOW,KY,42141,0,3,1970-01-01 00:00:00.000200204,2,...,1,202112.0,12,0.0,0.0,0.0,K26,KY CLASSIC ALPACA SHOW,Food & Nutrition,8778


In [125]:
# Creating a smaller dataset to use in merge with second dataset later
charites_by_zip_df = charity_df.groupby(['ZIP'])['ZIP'].count().reset_index(name='NUMBER_OF_CHARITIES')
charites_by_zip_df.head()

,ZIP,NUMBER_OF_CHARITIES
0,40003,6
1,40004,117
2,40006,21
3,40008,13
4,40009,1


## Cleaning the Second Dataset

In [121]:
# dropping the columns I will not need for this project
income_df = income_df.drop(["GEO_ID"], axis=1)

# Renaming the Columns I do need
income_df = income_df.rename(columns=income_df.iloc[0]).loc[1:]
income_df.columns = income_df.columns.str.replace("[!!]", "_")
income_df.columns = income_df.columns.str.replace(" ", "_")

income_df.head()

C:\Users\rebec\AppData\Local\Temp\ipykernel_52024\1350496004.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  income_df.columns = income_df.columns.str.replace("[!!]", "_")


,Geographic_Area_Name,Estimate__Households__Total,Estimate__Households__Median_income_(dollars),Estimate__Households__Mean_income_(dollars)
1,ZCTA5 38079,1311,32629,54397
2,ZCTA5 40003,749,78068,84259
3,ZCTA5 40004,11732,61974,76107
4,ZCTA5 40006,1675,61838,67429
5,ZCTA5 40007,25,-,48472


In [122]:
# Extracting the Zip code from the Geographic Area Name Column

income_df[['Geographic_Area_Name', 'ZIP']] = income_df['Geographic_Area_Name'].str.split(' ', 1, expand=True)
income_df = income_df.drop(["Geographic_Area_Name"], axis=1)
income_df.head()

,Estimate__Households__Total,Estimate__Households__Median_income_(dollars),Estimate__Households__Mean_income_(dollars),ZIP
1,1311,32629,54397,38079
2,749,78068,84259,40003
3,11732,61974,76107,40004
4,1675,61838,67429,40006
5,25,-,48472,40007


## Merging the two DataFrames

In [127]:
merged_df = pd.merge(income_df,charites_by_zip_df)
merged_df.head()

,Estimate__Households__Total,Estimate__Households__Median_income_(dollars),Estimate__Households__Mean_income_(dollars),ZIP,NUMBER_OF_CHARITIES
0,749,78068,84259,40003,6
1,11732,61974,76107,40004,117
2,1675,61838,67429,40006,21
3,1333,50037,61121,40008,13
4,572,55117,51992,40009,1
